In [1]:
import os
import pandas as pd
import numpy as np
import itertools as it

In [8]:
sub = 'pilot-54'
ses = 'ses-01'
battery = 'fbirn'

main_dir = '/home/sshankar/ibc/analysis_pipeline/ibc_main/neurospin_data/info/'
# main_dir = '/home/sshankar/Documents/Data/'
datadir = os.path.join(main_dir, sub)
csv_file = 'extracted_rt-fingertap_sub-54_ses-02_2112021-1233.csv'
tsv_file = 'rt-fingertap_pilot-54_ses_02.tsv'

In [9]:
file_df = pd.read_csv(os.path.join(datadir, csv_file))

In [10]:
# In the csv file extracted from the E-Prime txt file, 
# run onset is the 'GetReady.OffsetTime' column of last row, 
# and all other columns of this row are empty.
run_start = (file_df.loc[~np.isnan(file_df['Introduction.OffsetTime'])])['Introduction.OffsetTime']
run_start = run_start.values[0]

In [11]:
run_start

66210.0

In [12]:
# Get trial_types names from Procedure. This is a block design experiment 
# but there are trials of fix and checkerboard within each block. Blocks are 
# separated with Procedure names of "fixationBlock" and "checkerBlock".
types = file_df['Procedure']
trials = types.str.contains("Proc", regex=False)
trial_types = types[trials==True] 
trial_types[trial_types=='TrialProc'] = "fingertap"
trial_types[trial_types=='DelayProc'] = "rest"

# Rest block onsets and durations
onset_rest = (file_df.loc[~np.isnan(file_df['delay2.OnsetTime'])])['delay2.OnsetTime']
dur_rest = (file_df.loc[~np.isnan(file_df['delay2.OffsetTime'])])['delay2.OffsetTime'] - \
            (file_df.loc[~np.isnan(file_df['delay2.OnsetTime'])])['delay2.OnsetTime']

# Task block onsets and durations
onset_task = []
offset_task = []
dur_task = []

# First finger tapping block
onset_task.append(file_df.loc[0]['RTmeasure1.OnsetTime'])

for idx in dur_rest.keys():
    onset_task.append(file_df.loc[idx+1]['RTmeasure1.OnsetTime'])
    offset_task.append(file_df.loc[idx-2]['RTmeasure1.OffsetTime'])
    
offset_task.append(file_df.iloc[-3]['RTmeasure1.OffsetTime'])
dur_task = np.asarray(offset_task)-np.asarray(onset_task)

# Combine the task and rest onsets and durations and set them in order
onset_task.extend(list(onset_rest))
sortidx = np.argsort(onset_task)

onsets = np.asarray(onset_task)[sortidx]
durations = np.concatenate((dur_task, dur_rest.values))[sortidx]

In [13]:
# Create a DataFrame to save to events file
df = pd.DataFrame({'onset':(onsets-run_start)/1000, 'duration':durations/1000, 'trial_type':trial_types})
df.to_csv(os.path.join(datadir, tsv_file), sep='\t', float_format='%0.3f', index=False)